In [1]:
# encoding=utf8

# Data Preparation of English GP-practices or educational purposes

"data-preparation-GP-prescribing":
This file creates smaller datasets from the UK prescribing datasets

# Data analysis and visualization of GP-prescribing data

Remark: This notebook is fully reproducible as the data is fully accessible via a URL. However, start this notebook only if you have got sufficient memory and computational power as each GP-prescribing file has 1.3 GB for each month.

If you do not have got enough memory, use the generated data in the provided folder which makes use only of a small percentage of the full dataset.

In [ ]:
#preliminaries
import pandas as pd
import numpy as np

%matplotlib inline
import random
import matplotlib.pyplot as plt

### Read in the practice information data file to a pandas dataframe

In [ ]:
num_months = int(input("\nPlease enter the number of months you would like to show in notebook: "))
print ('you entered', num_months)

In [ ]:
#generate filenames for pre-selected months
path = 'http://datagov.ic.nhs.uk/presentation/' 
cols2 = ['sha', 'pct', 'practice', 'bnf_code', 'bnf_name', 'items', 'nic', 'act_cost', 'quantity', 'period']

prefixes=[] 
files=[]
months=[]

import calendar 
for year in range(2016,2017): #only for one year 
    for i in range(1,13):
        
        prefixes.append(str(year) +'_' +str(i).zfill(2) + '_' + calendar.month_name[i] + '/') 
        files.append('T' + str(year) +str(i).zfill(2) + 'PDPI+BNFT.CSV')
        months.append(calendar.month_name[i])
        
#read all dfs

monthly_dfs = []

for i in range(0,num_months):
    print (path+ prefixes[i] + files[i])
    tmpDF = pd.read_csv(path + prefixes[i] + files[i], header=None, names=cols2, index_col=False, skiprows=1).rename(columns=lambda x: x.strip())
    
    monthly_dfs.append(tmpDF)

# Identify the five most expensive drugs (costs per drug) for practice 'THE DENSHAM SURGERY' in July 2016 and plot them in a bar chart with pandas

In [ ]:
jul2016 = monthly_dfs[6]  

In [ ]:
practice_code= 'A81001' #"THE DENSHAM SURGERY"
selected_surgery = "THE DENSHAM SURGERY"
densham_data = jul2016[jul2016['practice'] == practice_code].copy(deep=True)
densham_data.head()

### Calculate costs per drug and store the five most expensive

In [ ]:
densham_data['costs_per_drug'] =  densham_data['act_cost'] / densham_data['quantity']
densham_sorted=densham_data.sort_values('costs_per_drug', axis=0, ascending=False, kind='quicksort', na_position='last')
densham_top5 = densham_sorted.head(5)

### Bar Chart

In [ ]:
%matplotlib inline

ax=densham_top5[['bnf_name','costs_per_drug']].plot(kind='bar', x='bnf_name', y='costs_per_drug', color=['r', 'g', 'b', 'r', 'g', 'b', 'r'], title='top 5 costs for: '+ selected_surgery, figsize=(6,4),legend=True, fontsize=10)

ax.set_xlabel("bnf",fontsize=8)

ax.set_ylabel("costs per drug",fontsize=8)

# Task 1: Plot a bar chart with the ten most expensive drugs for the Densham Surgery, modify the colours of the bars, and update the labels

In [ ]:
densham_top10 = densham_sorted.head(10)

In [ ]:
ax=densham_top10[['bnf_name','costs_per_drug']].plot(kind='bar', x='bnf_name', y='costs_per_drug', color=['r', 'g', 'b', 'r', 'y', 'b', 'y'], title='top 5 costs for: '+ selected_surgery, figsize=(6,4),legend=True, fontsize=10)

ax.set_xlabel("bnf",fontsize=8)

ax.set_ylabel("costs per drug",fontsize=8)

### Line Chart: Identify the time series of quantity) for ‘Ibuprofen-Tab 200 mg’ in the last three months for the Densham surgery and plot this data in a line chart

In [ ]:
#helper function
def findQuantityDrug( drugName, currentdf, practice_code):
    tempdf= currentdf[(currentdf['practice'] == practice_code) &(currentdf['bnf_name'].str.contains(drugName))]
    quantityDrug = tempdf.iloc[0]['quantity']
    return int(quantityDrug);

In [ ]:
def getMonthlyDrugPrescriptions(drugName):
    
    monthly_drug=[]
    
    for i in range(0,num_months):
        quantity = findQuantityDrug(drugName, currentdf=monthly_dfs[i], practice_code='A81001')
        monthly_drug.append(quantity)
        
    return monthly_drug

In [ ]:
drugName = "Ibuprofen_Tab 200mg"
monthly_IBUprofen = getMonthlyDrugPrescriptions(drugName)

# Line Chart

In [ ]:
#let's look at annual line chart of prescriptions per month in GP-practice
df2 = pd.DataFrame({'months' : pd.Categorical(months),
   'prescriptions per month' : pd.Series(monthly_IBUprofen, dtype='int32')})

titles="number of " + drugName + " prescriptions per month in GP-practice: " + selected_surgery         
ax0=df2.plot(kind='line', x='months', y='prescriptions per month', figsize=(8,6), title=titles, stacked=False)
ax0.grid() #adding a grid
ax0.legend([drugName])

# Task 2: Visualize "Cetirizine HCl_Tab 10mg"

In [ ]:
#solution
drugName = "Cetirizine HCl_Tab 10mg"

monthly_Ceterizine = getMonthlyDrugPrescriptions(drugName)

df2 = pd.DataFrame({'months' : pd.Categorical(months),
   'prescriptions per month' : pd.Series(monthly_Ceterizine, dtype='int32')})

titles="number of " + drugName + " prescriptions per month in GP-practice: " + selected_surgery         
ax0=df2.plot(kind='line', x='months', y='prescriptions per month', 
             figsize=(8,6), title=titles, stacked=False)
ax0.grid() #adding a grid
ax0.legend([drugName])

# Task 3: Visualize any medication of your interest

In [ ]:
#solution
drugName = "Atorvastatin_Tab 40mg"

monthly_Atorvastatin = getMonthlyDrugPrescriptions(drugName)

df2 = pd.DataFrame({'months' : pd.Categorical(months),
   'prescriptions per month' : pd.Series(monthly_Atorvastatin, dtype='int32')})

titles="number of " + drugName + " prescriptions per month in GP-practice: " + selected_surgery         
ax0=df2.plot(kind='line', x='months', y='prescriptions per month', 
             figsize=(8,6), title=titles, stacked=False)
ax0.grid() #adding a grid
ax0.legend([drugName])

# Identify how often (Atorvastatin_Tab 40mg, Simvastatin_Tab 40mg, Rosuvastatin Calc_Tab 40mg) were prescribed from Feb - August 2016 in England and plot this data in a stacked bar chart

In [ ]:
def findTotalCostsDrugUK( drugName, currentdf):
    return currentdf[currentdf['bnf_name'].str.contains(drugName)].act_cost.sum()

drugs = ["Atorvastatin_Tab 40mg", "Simvastatin_Tab 40mg"]

In [ ]:
def monthly_statins(drug, start_month, end_month):
    
    statin_monthly=[]
    for i in range(start_month, end_month+1):
        statin_monthly.append(findTotalCostsDrugUK(drug, monthly_dfs[i]))
    
    return statin_monthly

In [ ]:
atorvastatin=monthly_statins(drugs[0], 1, 7) #from January to August
simvastatin=monthly_statins(drugs[1], 1, 7)

In [ ]:
df3 = pd.DataFrame({'months' : pd.Categorical(months[1:8]), #Feb to August
                    'Atorvastatin' : pd.Series(atorvastatin,dtype='int64'),
                    'Simvastatin' : pd.Series(simvastatin,dtype='int64')})

In [ ]:
df3

In [ ]:
ax1 = df3.plot(kind='bar', x='months', title="bar chart: spendings for statins per months",stacked=True, color=['r', 'g', 'b'], figsize=(8,6), legend=True, fontsize=8)
ax1.set_xlabel("month",fontsize=12)
ax1.set_ylabel("costs",fontsize=12)
ax1.legend(loc='lower right')

# Task 5: Create a stacked bar chart from Jul - December 2016 and include "Rosuvastatin Calc_Tab 40mg"

In [ ]:
#solution
drugs = ["Atorvastatin_Tab 40mg", "Simvastatin_Tab 40mg", "Rosuvastatin Calc_Tab 40mg"]

atorvastatin=monthly_statins(drugs[0], 6, 11) #from January to December
simvastatin=monthly_statins(drugs[1], 6, 11)
Rosuvastatin=monthly_statins(drugs[2], 6, 11)

df3 = pd.DataFrame({'months' : pd.Categorical(months[6:12]), #Jul to Dec 2016
                    'Atorvastatin' : pd.Series(atorvastatin,dtype='int64'),
                    'Simvastatin' : pd.Series(simvastatin,dtype='int64'),
                    'Rosuvastatin' : pd.Series(Rosuvastatin,dtype='int64')})

In [ ]:
df3

In [ ]:
ax1 = df3.plot(kind='bar', x='months', title="bar chart: spendings for statins per months",stacked=True, color=['r', 'g', 'b'], figsize=(8,6), legend=True, fontsize=8)
ax1.set_xlabel("month",fontsize=12)
ax1.set_ylabel("costs",fontsize=12)
ax1.legend(loc='lower right')

# Scatterplot

In [ ]:
#Numbers of Patients Registered at a GP Practice
url_reg_pat= 'http://digital.nhs.uk/media/29065/Numbers-of-Patients-Registered-at-a-GP-Practice-July-2016-GP/Any/gp-reg-patients-prac-quin-age.csv'

gp_counts = pd.read_csv(url_reg_pat).rename(columns=lambda x: x.strip())

In [ ]:
total_costs_practices =pd.DataFrame(jul2016.groupby(jul2016.practice).apply(lambda subf: subf['act_cost'].sum()))
total_costs_practices.columns = ['total_costs_per_practice']
# Reseting the index
total_costs_practices.reset_index(inplace=True)
total_costs_practices.head()

In [ ]:
merged = pd.merge(total_costs_practices, gp_counts[['GP_PRACTICE_CODE', 'Total_All']], left_on='practice' , right_on='GP_PRACTICE_CODE')
ax3=merged.plot(kind='scatter', x='total_costs_per_practice', y='Total_All', title='Scatter plot: monthly total spending / registered patients', figsize=(8,6))
ax3.set_xlabel("monthly total spending per surgery",fontsize=12)
ax3.set_ylabel("total number of registered patients",fontsize=12)
ax3.set_xlim(0, 450000)
ax3.set_ylim(0, 40000)

# Task 6: Redraw the previous scatterplot and modify the max-value of x and y-axis 

In [ ]:
#solution

from math import floor
max_x = floor(merged.total_costs_per_practice.max())
max_y = floor(merged.Total_All.max())

ax4=merged.plot(kind='scatter', x='total_costs_per_practice', y='Total_All', title='Scatter plot: monthly total spending / registered patients', 
                color='royalblue', figsize=(8,6))
ax4.set_xlabel("monthly total spending per surgery",fontsize=12)
ax4.set_ylabel("total number of registered patients",fontsize=12)
ax4.set_xlim(0, max_x)
ax4.set_ylim(0, max_y)

# Scatterplot with correlation

In [ ]:
# plot those data points
#fig, ax = plt.subplots()
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(8, 6))
ax=fig.add_subplot(111)
x= merged['total_costs_per_practice']
y = merged['Total_All']
ax.scatter(x, y, color='royalblue')
fit = np.polyfit(x, y, deg=1)
ax.plot(x, fit[0] * x + fit[1], color='red')
ax.set_title('Scatter plot: monthly total spending / registered patients')
ax.set_xlabel("monthly total spending per surgery",fontsize=12)
ax.set_ylabel("total number of registered patients",fontsize=12)
max_x = floor(merged.total_costs_per_practice.max())
max_y = floor(merged.Total_All.max())
ax.set_xlim(0, max_x)
ax.set_ylim(0, max_y)

# Histogram

In [ ]:
cost_patient_plot = merged
cost_patient_plot['cost_per_patient'] = cost_patient_plot['total_costs_per_practice'] / cost_patient_plot['Total_All'] 

x = cost_patient_plot.cost_per_patient.values

plt.hist(x,3500)
plt.xlabel("Monthly prescription spending per patient")
plt.ylabel("Frequency")

min_x = floor(cost_patient_plot['cost_per_patient'].quantile(.01))
max_x = floor(cost_patient_plot['cost_per_patient'].quantile(.99))
plt.xlim(min_x, max_x) #do not show extreme outliers
plt.title("GP practice monthly total prescription spending per registered patient")
plt.show()

# Plot of google flu data

In [ ]:
url_au = 'https://www.google.org/flutrends/about/data/flu/au/data.txt'

#set header=8 to properly extract the data
australiaFluTrends = pd.read_csv(url_au, sep=',', header = 8)
australiaFluTrends['Date'] = pd.to_datetime(australiaFluTrends['Date'])

australiaFluTrends.head(5)

In [ ]:
australiaFluTrends.head()

In [ ]:
#Store the url of the database in a variable
url_ca = 'https://www.google.org/flutrends/about/data/flu/ca/data.txt'

#set header=8 to properly extract the data
canadaFluTrends = pd.read_csv(url_ca, sep=',', header=8)
canadaFluTrends['Date'] = pd.to_datetime(canadaFluTrends['Date'])

canadaFluTrends.head(5)

In [ ]:
%matplotlib inline
#Plot and store the flu trens for Canada

ax = canadaFluTrends.plot(legend ='left', x='Date', y = 'Canada', figsize=(15, 6), grid=True)
#Plot the flu trend for Australia; ax=ax plots the chart into the previous

australiaFluTrends.plot(x='Date', y = 'Australia' , ax=ax)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig_ca = canadaFluTrends.plot(x='Date', figsize=(10,8))
fig_ca.set_title('The trend of flu search activity in Canada',fontsize=15)
fig_ca.set_ylabel('Frequency',fontsize=15)
fig_ca.set_xlabel('Year',fontsize=15)

# Task: Plot the chart for Kazakhstan, China, Japan, Korea, Russia, Australia

In [ ]:
#solution
import matplotlib.pyplot as plt
%matplotlib inline
fig_au = australiaFluTrends.plot(x='Date', figsize=(10,8))
fig_au.set_title('The trend of flu search activity in Canada',fontsize=15)
fig_au.set_ylabel('Frequency',fontsize=15)
fig_au.set_xlabel('Year',fontsize=15)